# Efektywna rekonstrukcja zdjęć kotów za pomocą wariacyjnych autoenkoderów z pomocą Self Supervised Learning

## 1. Wprowadzenie

### 1.1 Opis problemu

### 1.2 Opis danych wejściowych

### 1.3 Opis danych wyjściowych

### 1.4 Hipoteza badawcza

## 2. Opis wybranych modeli oraz metod

### 2.1 Wariacyjne autoenkodery

#### 2.1.1 VAE - Variational Autoencoder

#### 2.1.2 VQ-VAE - Vector Quantized Variational Autoencoder

#### 2.1.3 VAE + Tree VI - Variational Autoencoder with Tree Variational Inference

#### 2.1.4 A-VAE / VAE-GAN  - Adversarial Variational Autoencoder

### 2.2 Self Supervised Learning

#### 2.2.1 BYOL - Bootstrap Your Own Latent

#### 2.2.2 SimCLR - Simple Framework for Contrastive Learning of Visual Representations

#### 2.2.3 Barlow Twins

## 3. Opis eksperymentów

### 3.1 Jak przygotowano zbiór danych?

### 3.2 Jakie metryki będą stosowane?

### 3.3 Badane hiperparametry modeli i metod

## 4. Wyniki eksperymentów

### 4.1 Część wariacyjnych autoenkoderów


#### 4.1.1 Baseline - VAE

#### 4.1.2 Wyniki VQ-VAE

#### 4.1.3 Wyniki VAE + Tree VI

#### 4.1.4 Wyniki A-VAE / VAE-GAN

### 4.2 Część Self Supervised Learning

#### 4.2.1 Wyniki BYOL oraz BYOL + VAE

#### 4.2.2 Wyniki Barlow Twins oraz Barlow Twins + VAE

#### 4.2.3 Wyniki SIM CLR oraz SIM CLR + VAE

## 5. Wnioski

### 5.1 Podsumowanie wyników

### 5.2 Analiza uzyskanych wyników w porównaniu z SOTA